<a href="https://colab.research.google.com/github/raphaelp-silva/GNN_Pytorch_geometric/blob/main/loading_a_graph_and_training_a_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.3 MB/s eta 0:00:00


In [19]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch.nn as nn

In [13]:
# Carregando o grafo salvo:
graph = torch.load('/content/obesity_graph_data-3.pt', weights_only=False)

# Acessando as variáveis do grafo carregado:
print(graph.x)  # Features dos nós
print(graph.edge_index)  # Arestas do grafo
print(graph.y)

tensor([[-1.2729, -0.6167, -0.8956,  ..., -0.0841,  0.6002, -0.1758],
        [-1.2729, -0.6167, -1.9360,  ..., -0.0841,  0.6002, -0.1758],
        [ 0.7856, -0.3259,  0.9770,  ..., -0.0841,  0.6002, -0.1758],
        ...,
        [-1.2729, -0.6167,  0.4568,  ..., -0.0841,  0.6002, -0.1758],
        [-1.2729, -0.6167,  1.0810,  ..., -0.0841,  0.6002, -0.1758],
        [-1.2729, -0.6167,  1.0810,  ..., -0.0841,  0.6002, -0.1758]])
tensor([[   0,    0,    0,  ..., 1566, 1566, 1566],
        [ 859,  721,   45,  ..., 1549, 1541, 1563]])
tensor([[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.]])


In [20]:
graph = Data(x = graph.x, edge_index=graph.edge_index, y=graph.y)
print(graph)

Data(x=[1567, 26], edge_index=[2, 23505], y=[1567, 6])


In [31]:
class GNN(torch.nn.Module):
  def __init__(self):

    super(GNN, self).__init__()

    self.conv1 = GCNConv(26, 128)
    self.conv2 = GCNConv(128, 64)
    self.conv3 = GCNConv(64, 32)
    self.fc = nn.Linear(32, 6)

  def forward(self, data):
    x = data.x
    edge_index = data.edge_index

    x = torch.relu(self.conv1(x, edge_index))
    x = torch.relu(self.conv2(x, edge_index))
    x = torch.relu(self.conv3(x, edge_index))
    x = self.fc(x)

    return torch.softmax(x, dim=1)

In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN().to(device)
data = graph.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(500):
  optimizer.zero_grad()
  out = model(data)
  loss = criterion(out, data.y)
  loss.backward()
  optimizer.step()

  if epoch % 10 == 0:
    print(f'Epoch: {epoch}, Loss: {loss.item():.4f}')


Epoch: 0, Loss: 1.7919
Epoch: 10, Loss: 1.4850
Epoch: 20, Loss: 1.4314
Epoch: 30, Loss: 1.4115
Epoch: 40, Loss: 1.3987
Epoch: 50, Loss: 1.3835
Epoch: 60, Loss: 1.3668
Epoch: 70, Loss: 1.3574
Epoch: 80, Loss: 1.3514
Epoch: 90, Loss: 1.3437
Epoch: 100, Loss: 1.3441
Epoch: 110, Loss: 1.3392
Epoch: 120, Loss: 1.3392
Epoch: 130, Loss: 1.3333
Epoch: 140, Loss: 1.3309
Epoch: 150, Loss: 1.3352
Epoch: 160, Loss: 1.3314
Epoch: 170, Loss: 1.3296
Epoch: 180, Loss: 1.3329
Epoch: 190, Loss: 1.3284
Epoch: 200, Loss: 1.3358
Epoch: 210, Loss: 1.3260
Epoch: 220, Loss: 1.3287
Epoch: 230, Loss: 1.3285
Epoch: 240, Loss: 1.3217
Epoch: 250, Loss: 1.3273
Epoch: 260, Loss: 1.3309
Epoch: 270, Loss: 1.3201
Epoch: 280, Loss: 1.3177
Epoch: 290, Loss: 1.3200
Epoch: 300, Loss: 1.3191
Epoch: 310, Loss: 1.3208
Epoch: 320, Loss: 1.3165
Epoch: 330, Loss: 1.3201
Epoch: 340, Loss: 1.3160
Epoch: 350, Loss: 1.3134
Epoch: 360, Loss: 1.3166
Epoch: 370, Loss: 1.3139
Epoch: 380, Loss: 1.3142
Epoch: 390, Loss: 1.3175
Epoch: 400,